### Start with pretrained model

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import pipeline
import numpy as np
import evaluate

In [16]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# Load the dataset with a specified encoding
file_path = '/Users/lukishyadav/Desktop/engineering/case_studies/ner_casestudy/data/ner_dataset.csv'  # Replace with your file path
data = pd.read_csv(file_path, encoding='ISO-8859-1')


In [18]:
# # Drop rows with NaN values
data = data.dropna(subset=['Word','POS','Tag'])

In [19]:
# Group the data by sentences
data['Sentence #'] = data['Sentence #'].ffill()  # Fill forward to propagate sentence IDs
sentences = data.groupby('Sentence #').apply(lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                                                      s['POS'].values.tolist(),
                                                                                      s['Tag'].values.tolist())])


In [20]:
# Convert the groupby object to a list of sentences
sentences = [s for s in sentences]


In [21]:
# Split the dataset into training, validation, and test sets (20% for test)
train_sentences, test_sentences = train_test_split(sentences, test_size=0.20, random_state=42)
train_sentences, val_sentences = train_test_split(train_sentences, test_size=0.25, random_state=42)  # 0.25 * 0.80 = 0.20


In [22]:
# Convert to Hugging Face Datasets format
def convert_to_dict(sentences):
    words = [[word for word, pos, tag in sentence] for sentence in sentences]
    pos_tags = [[pos for word, pos, tag in sentence] for sentence in sentences]
    ner_tags = [[tag for word, pos, tag in sentence] for sentence in sentences]
    return {"tokens": words, "pos_tags": pos_tags, "ner_tags": ner_tags}


In [23]:
train_data = convert_to_dict(train_sentences)
val_data = convert_to_dict(val_sentences)
test_data = convert_to_dict(test_sentences)


In [25]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model_checkpoint="nlpso/m1_ind_layers_ref_ptrn_cmbert_iob2_level_2"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
pre_trained_model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

/opt/anaconda3/envs/gen_3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:

nlp = pipeline("ner", model=pre_trained_model, tokenizer=tokenizer)
example = "Ronaldo is Goat!"

ner_results = nlp(example)
print(ner_results)


[]


In [27]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [28]:
nlp(test_data[])

SyntaxError: invalid syntax (1628895100.py, line 1)

In [31]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import pipeline
import numpy as np
import evaluate
import torch

# Ensure MLflow directory exists
mlruns_dir = '/Users/lukishyadav/Desktop/engineering/case_studies/ner_casestudy/mlruns'
if not os.path.exists(mlruns_dir):
    os.makedirs(mlruns_dir)

import mlflow

mlflow.set_tracking_uri('file:///Users/lukishyadav/Desktop/engineering/case_studies/ner_casestudy/mlruns')

experiment_id = mlflow.create_experiment('NER_Casestudy_Experiment')

# Create or get the experiment
experiment_name = "NER_Casestudy_Experiment"
mlflow.set_experiment(experiment_name)


# Load the dataset with a specified encoding
file_path = '/Users/lukishyadav/Desktop/engineering/case_studies/ner_casestudy/data/ner_dataset.csv'  # Replace with your file path
data = pd.read_csv(file_path, encoding='ISO-8859-1')

# Drop rows with NaN values
data = data.dropna()

# Group the data by sentences
data['Sentence #'] = data['Sentence #'].ffill()  # Fill forward to propagate sentence IDs
sentences = data.groupby('Sentence #').apply(lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                                                      s['POS'].values.tolist(),
                                                                                      s['Tag'].values.tolist())])
# Convert the groupby object to a list of sentences
sentences = [s for s in sentences]

# Split the dataset into training, validation, and test sets (20% for test)
train_sentences, test_sentences = train_test_split(sentences, test_size=0.20, random_state=42)
train_sentences, val_sentences = train_test_split(train_sentences, test_size=0.25, random_state=42)  # 0.25 * 0.80 = 0.20

# Convert to Hugging Face Datasets format
def convert_to_dict(sentences):
    words = [[word for word, pos, tag in sentence] for sentence in sentences]
    pos_tags = [[pos for word, pos, tag in sentence] for sentence in sentences]
    ner_tags = [[tag for word, pos, tag in sentence] for sentence in sentences]
    return {"tokens": words, "pos_tags": pos_tags, "ner_tags": ner_tags}

train_data = convert_to_dict(train_sentences)
val_data = convert_to_dict(val_sentences)
test_data = convert_to_dict(test_sentences)

# Create a dataset dictionary
dataset_dict = DatasetDict({
    'train': Dataset.from_dict(train_data),
    'validation': Dataset.from_dict(val_data),
    'test': Dataset.from_dict(test_data)
})

# Define unique tags
unique_tags = list(set(tag for doc in dataset_dict['train']['ner_tags'] for tag in doc))
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

# Tokenizer
model_checkpoint = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2id[label[word_idx]])
            else:
                label_ids.append(tag2id[label[word_idx]] if True else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(tokenize_and_align_labels, batched=True)

# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Model
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(unique_tags), ignore_mismatched_sizes=True)
model.classifier = torch.nn.Linear(model.classifier.in_features, len(unique_tags))
model.num_labels = len(unique_tags)

# Metrics
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Training arguments
args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start MLflow run
with mlflow.start_run() as run:
    # Train the model
    trainer.train()

    # Evaluate the model
    results = trainer.evaluate(tokenized_datasets["test"])

    # Log metrics to MLflow
    mlflow.log_metrics(results)

# Predict with the model
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
sample_text = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge."
ner_results = ner_pipeline(sample_text)
print(ner_results)



/opt/anaconda3/envs/gen_3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/28769 [00:00<?, ? examples/s]

Map:   0%|          | 0/9590 [00:00<?, ? examples/s]

Map:   0%|          | 0/9590 [00:00<?, ? examples/s]

/opt/anaconda3/envs/gen_3.8/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5397 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: Placeholder storage has not been allocated on MPS device!